In [1]:
from models.resnet import MyResNet
from data_utils import CustomDataset, get_transforms
from tqdm.notebook import tqdm
from torchvision.models import densenet121
import torch
import pandas as pd
import numpy as np

device = 'cuda:5'

In [2]:
checkpoint = 'glamorous-pyramid-63-17-0.8034'

In [3]:
def inference(model, dataloader, device='cpu'):
    preds = []
    targets = []
    model.eval()
    for x_batch, y_batch in tqdm(dataloader):
        data = x_batch.to(device)
        with torch.inference_mode():
            output = model(data)
            preds.append(output.detach().cpu())
        targets.append(y_batch)
            
    return torch.cat(preds), torch.cat(targets)

def predict(model_cls, checkpoint, batchsize=512):
    model = model_cls(densenet121())
    model.load_state_dict(torch.load(checkpoint))
    model.to(device)
#     model.base_model.classifier = torch.nn.Sequential()
    dataloader = torch.utils.data.DataLoader(CustomDataset('data/train.csv', get_transforms('train')), batch_size=batchsize, num_workers=8)
    preds = inference(model, dataloader, device)
    return preds

preds, targets = predict(MyResNet, f'checkpoints/{checkpoint}_model.pt')

  0%|          | 0/336 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [46]:
X_train, X_test, y_train, y_test = train_test_split(preds, targets, test_size=0.1, random_state=0xbebebe)

In [53]:
val = pd.read_csv('data/dev_val.csv')
val['Sex'] = val.Sex.apply(lambda x: int(x == 'Male'))

In [57]:
cv = KFold(10, shuffle=True, random_state=0xbebebe)
res = []
weights = []
biases = []

for tr, ts in cv.split(y_test):
    features = val[['Sex', 'Age']].values
    model = BinaryRelevance(LogisticRegression()).fit(np.hstack([X_test[tr], features[tr]]), y_test[tr])
    weights.append(np.vstack([x.coef_ for x in model.classifiers_]))
    biases.append(np.vstack([x.intercept_ for x in model.classifiers_]))
    res.append(roc_auc_score(y_test[ts], model.predict_proba(np.hstack([X_test[ts], features[ts]])).todense()))
    print(res[-1])

0.7942550995059536
0.8182695998406162


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7929420176459525
0.7955340726585587


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8057167185780797
0.8097339235557157


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7954949576630604
0.7997372777902418


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.803128188915959
0.7969088784722003


In [14]:
np.mean(biases, 0).flatten()

array([ 0.13252833, -0.04922415, -0.38749676, -0.05492893,  0.1900582 ])

In [12]:
np.mean(weights, 0)

array([[ 0.8884573 , -0.02050001,  0.06470138,  0.07708601, -0.03539958],
       [ 0.01634081,  0.90970256,  0.04924741, -0.03046782, -0.02745024],
       [ 0.00647937, -0.00445542,  0.83861348,  0.03517183,  0.01810227],
       [ 0.00894726, -0.00640146, -0.01189516,  0.85447432,  0.01298881],
       [-0.0050264 ,  0.00760905,  0.02597707, -0.05035251,  0.93361056]])

In [10]:
np.mean(res)

0.8013588835349263

In [170]:
model = BinaryRelevance(LogisticRegression(max_iter=)).fit(X_train, y_train)
roc_auc_score(y_test, model.predict_proba(X_test).todense())

AttributeError: 'BinaryRelevance' object has no attribute 'decision_function'

In [179]:
roc_auc_score(y_test, np.vstack([m.decision_function(X_test) for m in model.classifiers_]).T)

0.7950008737434862

In [171]:
model = RidgeClassifier().fit(X_train, y_train)
roc_auc_score(y_test, model.decision_function(X_test).todense())

ValueError: RidgeClassifier doesn't support multi-label classification

In [20]:
def inference(model, dataloader, device='cpu'):
    preds = []
    targets = []
    model.eval()
    for epoch in range(5):
        for x_batch, y_batch in tqdm(dataloader):
            data = x_batch.to(device)
            with torch.inference_mode():
                output = model(data)
                preds.append(output.detach().cpu())
            targets.append(y_batch)
            
    return torch.cat(preds), torch.cat(targets)

def predict(model_cls, checkpoint, batchsize=512):
    model = model_cls(densenet121())
    model.load_state_dict(torch.load(checkpoint))
    model.to(device)
#     model.base_model.classifier = torch.nn.Sequential()
    dataloader = torch.utils.data.DataLoader(CustomDataset('data/dev_val.csv', get_transforms('train')), batch_size=batchsize, num_workers=8)
    preds = inference(model, dataloader, device)
    return preds

val_preds, val_targets = predict(MyResNet, f'checkpoints/{checkpoint}_model.pt')

  0%|          | 0/34 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

  0%|          | 0/34 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

  0%|          | 0/34 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

  0%|          | 0/34 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

  0%|          | 0/34 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487: UserWarning: torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3668.)
  res = torch.lstsq(b_matrix, a_matrix)[0]
/opt/conda/lib/python3.6/site-packages/torchvision/transforms/functional.py:487

In [37]:
cv = KFold(5, shuffle=True, random_state=0xbebebe)
res = []
weights = []
biases = []

for tr, ts in cv.split(val_targets):
    model = BinaryRelevance(LogisticRegression()).fit(val_preds[tr], val_targets[tr])
    weights.append(np.vstack([x.coef_ for x in model.classifiers_]))
    biases.append(np.vstack([x.intercept_ for x in model.classifiers_]))
    res.append(roc_auc_score(val_targets[ts], model.predict_proba(val_preds[ts]).todense()))
    print(res[-1])
print(np.mean(res))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.7971654649984079
0.8039469994179018


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8029164492178305


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

0.8017715946022573
0.7957602459478771


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.801637384157246


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8053524441378691
0.8011126822909675
